In [8]:
import os
import pandas as pd

import yaml
from catboost import CatBoostRegressor
from sklearn.metrics import r2_score

import sys 
sys.path.append('../tools/')
from utils import create_experiment_folder
from dataloader import CustomDatasetRegression

import warnings
warnings.filterwarnings("ignore")

In [24]:
EXP_PATH = 'catboost/experiments'
ARGS = {
    'iterations': 100, 
    'learning_rate': 0.1,
    'depth': 7,
    'loss_function': 'RMSE',
    'verbose': False
}
DYNAMICS = [
    'linear',
    'linear_short',
    'steps_up',
    'steps_up_pulse',
    'steps_down',
    'steps_down_pulse'
]
SENSORS = ['R1','R2']#, 'R3', 'R4', 'R5', 'R6', 'R7', 'R8', 'R9', 'R10', 'R11', 'R12']
GASES = ['NO', 'CH4']#, 'H2S', 'SO2', 'HCOH', 'CO', 'H2', 'NO2', 'NH3']

In [3]:
def train_catboost(
        path, 
        exp_path, 
        dynamics, 
        sensors, 
        gases, 
        metric
    ):

    global ARGS
    exp_path = create_experiment_folder(exp_path)
    with open(os.path.join(exp_path, 'config.yaml'), 'w') as file:
        yaml.dump(ARGS, file, default_flow_style=False)

    metric_dict = {}
    for sensor in sensors:
        metric_dict[sensor] = pd.DataFrame(index=dynamics, columns=gases)

    for dynamic in dynamics:
        print(f'Dynamic {dynamic}')
        for gas in gases:
            dataset_train = CustomDatasetRegression(
                path=path,
                dynamic=dynamic, 
                gas=gas, 
                step='trn', 
                output_type='np'
            )
            dataset_valid = CustomDatasetRegression(
                path=path,
                dynamic=dynamic, 
                gas=gas, 
                step='vld', 
                output_type='np'
            )
            for sensor in sensors:
                try:
                    x_train, y_train = dataset_train.get_all_data(sensor, 'conc')
                    x_valid, y_valid = dataset_valid.get_all_data(sensor, 'conc')

                    model = CatBoostRegressor(**ARGS)
                    model.fit(x_train, y_train)

                    y_valid_pred = model.predict(x_valid)
                    metric_value = metric(y_valid, y_valid_pred)
                    metric_dict[sensor].loc[dynamic, gas] = metric_value

                    cb_model_name = f'cb_{sensor}_{dynamic}_{gas}.cbm'
                    model.save_model(os.path.join(exp_path, cb_model_name))
                except Exception as e:
                    print(f'Sensor {sensor} on {dynamic} dynamic of {gas} gas: {e}')
                    
    
    for sensor in sensors:
        metric_dict[sensor].to_csv(os.path.join(exp_path, f'{sensor}.csv'), index=False)

### Train exp_1

In [11]:
train_catboost(
    data_path='../data/exp1_melted/',
    exp_path='experiments/', 
    dynamics=DYNAMICS, 
    sensors=SENSORS, 
    gases=GASES, 
    metric=r2_score
)

Created folder: /home/chernov.kirill9/projects/gases/gases-sensors/catboost/experiments/exp_1
Dynamic linear
Dynamic linear_short
Dynamic steps_up
Dynamic steps_up_pulse
Dynamic steps_down
Dynamic steps_down_pulse


### Train exp_2

In [29]:
train_catboost(
    path='../data/exp2_melted/',
    exp_path='experiments/', 
    dynamics=DYNAMICS, 
    sensors=SENSORS, 
    gases=GASES, 
    metric=r2_score
)

Created folder: /home/chernov.kirill9/projects/gases/gases-sensors/catboost/experiments/exp_4
Dynamic linear
Sensor R1 on linear dynamic of NO gas
Sensor R2 on linear dynamic of NO gas
Sensor R3 on linear dynamic of NO gas
Sensor R4 on linear dynamic of NO gas
Sensor R5 on linear dynamic of NO gas
Sensor R6 on linear dynamic of NO gas
Sensor R7 on linear dynamic of NO gas
Sensor R8 on linear dynamic of NO gas
Sensor R9 on linear dynamic of NO gas
Sensor R10 on linear dynamic of NO gas
Sensor R11 on linear dynamic of NO gas
Sensor R12 on linear dynamic of NO gas
Sensor R1 on linear dynamic of CH4 gas
Sensor R2 on linear dynamic of CH4 gas
Sensor R3 on linear dynamic of CH4 gas
Sensor R4 on linear dynamic of CH4 gas
Sensor R5 on linear dynamic of CH4 gas
Sensor R6 on linear dynamic of CH4 gas
Sensor R7 on linear dynamic of CH4 gas
Sensor R8 on linear dynamic of CH4 gas
Sensor R9 on linear dynamic of CH4 gas
Sensor R10 on linear dynamic of CH4 gas
Sensor R11 on linear dynamic of CH4 gas
S

## Tests

In [5]:
def test_catboost(
        data_path,
        exp_path,
        results_path,
        dynamics,
        sensors,
        gases,
        metric
    ):
    results_path = os.path.join(results_path, data_path.split('/')[-1] + '_on_' + exp_path.split('/')[-1])
    if not os.path.exists(results_path):
        os.makedirs(results_path)

    metric_dict = {}
    for sensor in sensors:
        metric_dict[sensor] = pd.DataFrame(index=dynamics, columns=gases)

    for dynamic in dynamics:
        print(f'Dynamic {dynamic}')
        for gas in gases:
            dataset_test = CustomDatasetRegression(
                path=data_path,
                dynamic=dynamic,
                gas=gas,
                step='tst',
                output_type='np'
            )
            for sensor in sensors:
                try:
                    x_test, y_test = dataset_test.get_all_data(sensor, 'conc')

                    cb_model_name = f'cb_{sensor}_{dynamic}_{gas}.cbm'
                    model_path = os.path.join(exp_path, cb_model_name)

                    model = CatBoostRegressor()
                    model.load_model(model_path)

                    y_test_pred = model.predict(x_test)
                    metric_value = metric(y_test, y_test_pred)
                    metric_dict[sensor].loc[dynamic, gas] = metric_value
                except Exception as e:
                    print(f'Sensor {sensor} on {dynamic} dynamic of {gas} gas: {e}')

    for sensor in sensors:
        metric_dict[sensor].to_csv(os.path.join(results_path, f'{sensor}_test_metrics.csv'))

In [21]:
test_catboost(
        data_path='/home/chernov.kirill9/projects/gases/gases-sensors/data/exp2_melted',
        exp_path='/home/chernov.kirill9/projects/gases/gases-sensors/catboost/experiments/exp_1',
        results_path='/home/chernov.kirill9/projects/gases/gases-sensors/catboost/results',
        dynamics=DYNAMICS,
        sensors=SENSORS,
        gases=GASES,
        metric=r2_score
    )

Dynamic linear
Dynamic linear_short
Dynamic steps_up
Dynamic steps_up_pulse
Dynamic steps_down
Dynamic steps_down_pulse


In [37]:
test_catboost(
    data_path='/home/chernov.kirill9/projects/gases/gases-sensors/data/exp1_melted',
    exp_path='/home/chernov.kirill9/projects/gases/gases-sensors/catboost/experiments/exp_1',
    results_path='/home/chernov.kirill9/projects/gases/gases-sensors/catboost/results',
    dynamics=DYNAMICS,
    sensors=SENSORS,
    gases=GASES,
    metric=r2_score
)

Dynamic linear
Dynamic linear_short
Dynamic steps_up
Dynamic steps_up_pulse
Dynamic steps_down
Dynamic steps_down_pulse


In [36]:
test_catboost(
    data_path='/home/chernov.kirill9/projects/gases/gases-sensors/data/exp2_melted',
    exp_path='/home/chernov.kirill9/projects/gases/gases-sensors/catboost/experiments/exp_2',
    results_path='/home/chernov.kirill9/projects/gases/gases-sensors/catboost/results',
    dynamics=DYNAMICS,
    sensors=SENSORS,
    gases=GASES,
    metric=r2_score
)

Dynamic linear
Dynamic linear_short
Sensor R4 on linear_short dynamic of NO gas: catboost/libs/model/model_import_interface.h:19: Model file doesn't exist: /home/chernov.kirill9/projects/gases/gases-sensors/catboost/experiments/exp_2/cb_R4_linear_short_NO.cbm
Sensor R6 on linear_short dynamic of NO gas: catboost/libs/model/model_import_interface.h:19: Model file doesn't exist: /home/chernov.kirill9/projects/gases/gases-sensors/catboost/experiments/exp_2/cb_R6_linear_short_NO.cbm
Sensor R4 on linear_short dynamic of CH4 gas: catboost/libs/model/model_import_interface.h:19: Model file doesn't exist: /home/chernov.kirill9/projects/gases/gases-sensors/catboost/experiments/exp_2/cb_R4_linear_short_CH4.cbm
Sensor R4 on linear_short dynamic of H2S gas: catboost/libs/model/model_import_interface.h:19: Model file doesn't exist: /home/chernov.kirill9/projects/gases/gases-sensors/catboost/experiments/exp_2/cb_R4_linear_short_H2S.cbm
Sensor R6 on linear_short dynamic of H2S gas: catboost/libs/mode